# Evaluation criteria

The goal of this assignment is to get a view on your hands-on "data engineering" skills.  
At our company, our data scientists and engineers collaborate on projects.  
Your main focus will be creating performant & robust data flows.  
For a take-home-assignment, we cannot grant you access to our infrastructure.  
The assignement below measures your proficiency in general programming, data science & engineering tasks using python.  
Completion should not take more than half a day.

**We expect you to be proficient in:**
 * SQL queries (Sybase IQ system)
 * ETL flows (In collaboration with existing teams)
 * General python to glue it all together
 * Python data science ecosystem (Pandas + SKlearn)
 
**In this exercise we expect you to demonstrate your ability to / knowledge of:**
 * Building a data science runtime
 * PEP8 / Google python styleguide
 * Efficiently getting the job done
 * Choose meaningfull names for variables & functions
 * Writing maintainable code (yes, you might need to document some steps)
 * Help a data scientist present interactive results.
 * Offer predictions via REST api

# Setting-up a data science workspace

We allow you full freedom in setting up a data science runtime.  
The main objective is having a runtime where you can run this notebook and the code you will develop.  
You can choose for a local setup on your pc, or even a cloud setup if you're up for it.   

**In your environment, you will need things for:**
 * https request
 * python3 (not python2 !!)
 * (geo)pandas
 * interactive maps (e.g. folium, altair, ...)
 * REST apis
 
**Deliverables we expect**:
 * notebook with the completed assignment
 * list of packages for your runtime (e.g. yml or txt file)
 * evidence of a working API endpoint

# Importing packages

We would like you to put all your import statements here, together in 1 place.  
Before submitting, please make sure you remove any unused imports :-)  

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import json
import unittest
from joblib import load
from math import radians, cos, sin, asin, sqrt

# Data ingestion exercises

## Getting store location data from an API

**Goal:** Obtain a pandas dataframe  
**Hint:** You will need to normalise/flatten the json, because it contains multiple levels  
**API call:** https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places  

In [ ]:
def get_clp_places(url):
    with urllib.request.urlopen(url) as req:
        data = json.loads(req.read().decode())
    df = pd.json_normalize(data)
    return df

df_clp = get_clp_places("https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places")
df_clp.head(10)

### Quality checks

We would like you to add several checks on this data based on these constraints:  
 * records > 200
 * latitude between 49 and 52
 * longitude between 2 and 7
 
We dont want you to create a full blown test suite here, we're just gonna use 'asserts' from unittest

In [ ]:
tc = unittest.TestCase('__init__')
# records > 200
tc.assertGreater(len(df_clp.index), 200, "There should be more than 200 records")
# latitude between 49 and 52
tc.assertTrue(
    (df_clp["geoCoordinates.latitude"].min() >= 49) & (df_clp["geoCoordinates.latitude"].max() <= 52), 
    "All stores should have a latitude between 49 and 52"
)
# longitude between 2 and 7
tc.assertTrue(
    (df_clp["geoCoordinates.longitude"].min() >= 2) & (df_clp["geoCoordinates.longitude"].max() <= 7), 
    "All stores should have a longitude between 2 and 7"
)

### Feature creation

Create a new column "antwerpen" which is 1 for all stores in Antwerpen (province) and 0 for all others 

In [ ]:
df_clp["antwerpen"] = np.where(df_clp["address.postalcode"].astype(int).between(2000, 2999), 1, 0)
df_clp["antwerpen"].value_counts()

## Predict used car value

A datascientist in our team made a basic model to predict car prices.  
The model was saved to disk ('lgbr_cars.model') using joblib's dump fuctionality.  
Documentation states the model is a LightGBM Regressor, trained using the sk-learn api.  

**As engineer, your task it to expose this model as REST-api.** 

First, retrieve the model via the function below.  
Change the path according to your setup.  

In [ ]:
def retrieve_model(path):
    trained_model = load(path)
    return trained_model

lgbr_cars = retrieve_model("lgbr_cars.model")

tc.assertEqual(str(type(lgbr_cars)),"<class 'lightgbm.sklearn.LGBMRegressor'>", type(lgbr_cars))

Now you have your trained model, lets do a functional test based on the parameters below.  
You have to present the parameters in this order.  

* vehicleType: coupe
* gearbox: manuell
* powerPS: 190
* model: NaN
* kilometer: 125000
* monthOfRegistration: 5 
* fuelType: diesel
* brand: audi

Based on these parameters, you should get a predicted value of 14026.35068804
However, the model doesnt accept string inputs, see the integer encoding below:

In [ ]:
model_test_input = [[3,1,190,-1,125000,5,3,1]]

In [ ]:
def make_prediction(trained_model, single_input):
    predicted_value = trained_model.predict(single_input)
    return predicted_value[0]

predicted_value = make_prediction(lgbr_cars, model_test_input)

tc.assertAlmostEqual(predicted_value, 14026.35, places=2)
# just to make sure, we also assert our test data for the following excericise
tc.assertAlmostEqual(make_prediction(lgbr_cars, [[-1,1,0,118,150000,0,1,38]]), 13920.70, places=2)

Now you got this model up and running, we want you to **expose it as a rest api.**  
We don't expect you to set up any authentication.  
We're not looking for beautiful inputs, just make it work.  
**Building this endpoint should NOT be done in a notebook, but in proper .py file(s)**

Once its up and running, use it to predict the following input:
* [-1,1,0,118,150000,0,1,38] ==> prediction should be 13920.70

## Geospatial data exercise
The goal of this exercise is to read in some data from a shape file and visualize it on a map
- The map should be dynamic. I want to zoom in and out to see more interesting aspects of the map
- We want you to visualize the statistical sectors within a distance of 2KM of your home location.

Specific steps to take:
- Read in the shape file
- Transform to WGS coordinates
- Create a distance function (Haversine)
- Create variables for home_lat, home_lon and perimeter_distance
- Calculate centroid for each nis district
- Calculate the distance to home for each nis district centroid 
- Figure out which nis districts are near your home
- Create dynamic zoomable map
- Visualize the nis districts near you (centroid <2km away), on the map


In [ ]:
# Some imports to help you along the way
import geopandas as gpd
import folium # you can use any viz library you prefer

In [9]:
# part 1: Reading in the data
# get this file from https://statbel.fgov.be/sites/default/files/files/opendata/Statistische%20sectoren/sh_statbel_statistical_sectors_20200101.shp.zip 
df = gpd.read_file('sh_statbel_statistical_sectors_20200101.shp')
df = df.to_crs("EPSG:4326") # change projection to wgs84 

# One of the data scientists discovered stackoverflow ;-) and copypasted something from https://gis.stackexchange.com/questions/166820/geopandas-return-lat-and-long-of-a-centroid-point
# A data science engineer should be able to speed this next code up
df["lon"] = df.geometry.centroid.x
df["lat"] = df.geometry.centroid.y

C:\Users\kevin\AppData\Local\Temp/ipykernel_17884/2146413365.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df["lon"] = df.geometry.centroid.x
C:\Users\kevin\AppData\Local\Temp/ipykernel_17884/2146413365.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df["lat"] = df.geometry.centroid.y


In [10]:
# Let's create some variables to indicate the location of your interest 
home = (51.101129, 4.369611)
colruyt_HQ = (50.72953273040411, 4.222002043806561)
perimeter_distance = 2 # km

In [11]:
# ripped from https://stackoverflow.com/a/4913653
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

# https://stackoverflow.com/a/51431891
def haversine_vectorized(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """

    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    dlat = np.subtract(lat2, lat1)

    a = np.add(np.power(np.sin(np.divide(dlat, 2)), 2),  
                          np.multiply(np.cos(lat1), 
                                      np.multiply(np.cos(lat2), 
                                                  np.power(np.sin(np.divide(dlon, 2)), 2))))
    c = np.multiply(2, np.arcsin(np.sqrt(a)))
    r = 6371

    return c*r

Next, implement some sanity checks for your distance function 

In [16]:
# implement sanity checks here
lyon = (45.7597, 4.8422)
paris = (48.8567, 2.3508)
expectedDistanceLyonParis = 392.2172595594006
tc.assertEqual(haversine(0, 0, 0, 0), 0)
tc.assertAlmostEqual(haversine(lyon[1], lyon[0], paris[1], paris[0]), expectedDistanceLyonParis, places=2)
tc.assertAlmostEqual(haversine(paris[1], paris[0], lyon[1], lyon[0]), expectedDistanceLyonParis, places=2)
tc.assertAlmostEqual(haversine(home[1], home[0], colruyt_HQ[1], colruyt_HQ[0]), 42.6, places=2)

tc.assertEqual(haversine_vectorized(0, 0, 0, 0), 0)
tc.assertAlmostEqual(haversine_vectorized(lyon[1], lyon[0], paris[1], paris[0]), expectedDistanceLyonParis, places=2)
tc.assertAlmostEqual(haversine_vectorized(paris[1], paris[0], lyon[1], lyon[0]), expectedDistanceLyonParis, places=2)
tc.assertAlmostEqual(haversine_vectorized(home[1], home[0], colruyt_HQ[1], colruyt_HQ[0]), 42.6, places=2)

print("The distance from home to Colruyt HQ is", round(haversine(home[1], home[0], colruyt_HQ[1], colruyt_HQ[0]), 2), "km")

# benchmark both functions
from time import perf_counter as time_perf_counter
start_normal = time_perf_counter()
_ = [haversine(home[1], home[0], lon, lat) for lon, lat in zip(df["lon"], df["lat"])]
end_normal = start_vectorized = time_perf_counter()
_ = haversine_vectorized(home[1], home[0], df["lon"], df["lat"])
end_vectorized = time_perf_counter()
print("havesine() for each row took:", end_normal - start_normal)
print("haversine_vectorized() took:", end_vectorized - start_vectorized)

The distance from home to Colruyt HQ is 42.6 km
havesine() for each row took: 0.028212400000029447
haversine_vectorized() took: 0.0034871999999950276


Now, create a dynamical map 

In [ ]:
# implementation of the map goes here
foliumMap = folium.Map(location=[home[0], home[1]], zoom_start=14)

# Add home
folium.Marker(location=[home[0], home[1]], popup="Home", icon=folium.Icon(icon="home", color="green")).add_to(foliumMap)

# Add Colruyt HQ
folium.Marker(location=[colruyt_HQ[0], colruyt_HQ[1]], popup="ColruytHQ", icon=folium.Icon(color="red")).add_to(foliumMap)

# Add all areas close to home
def add_geometry_to_folium(geometry, lat, lon, popup):
    sim_geo = gpd.GeoSeries(geometry)#.simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'blue'})
    geo_j.add_to(foliumMap)

    folium.Marker(location=[lat, lon], popup=popup).add_to(foliumMap)

close_to_home = haversine_vectorized(home[1], home[0], df["lon"], df["lat"]) <= perimeter_distance
_ = [add_geometry_to_folium(geometry, lat, lon, popup) for geometry, lat, lon, popup in df[close_to_home][["geometry", "lat", "lon", "T_SEC_NL"]].to_numpy()]

# Add all Colruyt supermarkets from df_clp
for _, r in df_clp.iterrows():
    folium.Marker(location=[r['geoCoordinates.latitude'], r['geoCoordinates.longitude']], icon=folium.Icon(color="orange"), popup=r["commercialName"]).add_to(foliumMap)

foliumMap